In [144]:
import os
import string
from pydub import AudioSegment
from pydub.silence import split_on_silence
import youtube_dl
import sqlite3
import logging
import re

In [6]:
def save_audio_chunks(audio_file, target_dir, min_silence_len=600, 
                      silence_thres=-40, keep_silence=300, format="mp3"):
    outpath = os.path.join(target_dir, os.path.split(audio_file)[-1])
    outpath = outpath.split(".")[0] + "_{:2d}" + ".mp3"

    sound = AudioSegment.from_mp3(audio_file)
    chunks = split_on_silence(sound, min_silence_len=min_silence_len,
                              silence_thresh=silence_thres)
#     for i, chunk in enumerate(chunks):
#         chunk.export(outpath.format(i), format=format)
    return chunks

In [7]:
# test_file = "/Users/kzecchini/Downloads/UMvC3 Voice Clips (English) - Vergil.mp3"
test_file = "./Marvel vs Capcom 3 - Zero Sound Clips (English)-P23PzmEpwk0.mp3"
chunks = save_audio_chunks(test_file, "./data/test")

In [9]:
chunks[-6]

In [11]:
sound = AudioSegment.from_mp3(test_file)

In [194]:
sound.fade_out(10000)

In [14]:
sound[:5000]

In [170]:
def get_video_id(url):
    video_id = None
    pattern = "^.*(youtu.be\/|v\/|e\/|u\/\w+\/|embed\/|v=)([^#\&\?]*).*"
    m = re.match(pattern, url)
    if m:
        video_id = m[2]
    return video_id

In [171]:
def get_files(path):
    filenames = []
    for filename in os.listdir(path):
        rel_path = os.path.join(path, filename)
        if os.path.isfile(rel_path):
            filenames.append(rel_path)
    return filenames

In [182]:
def check_videos(url, path):
    filenames = get_files(path)
    filename_ids = {os.path.split(filename)[-1].split("_")[0]: filename for filename in filenames}
    video_id = get_video_id(url)
    for fid in filename_ids.keys():
        if fid == video_id:
            return filename_ids[video_id]

In [183]:
check_videos(test_full, './')

'./P23PzmEpwk0_1561827823.mp3'

In [224]:
logging.basicConfig()
logger = logging.getLogger()


ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'logger': logger,
    'outtmpl': '%(id)s_%(epoch)s.%(ext)s',
    'progress_hooks': [my_hook],
}


def my_hook(d):
    if d['status'] == 'finished':
        logging.info('Done downloading, now converting ...')

        
def download_video(url, ydl_opts, fpath):
    curr_path = check_videos(url, fpath) 
    
    if curr_path is not None:
        logging.info("already found youtube file")
        return curr_path
    
    else:
        try:
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(url, download=False)
                ydl.download([url])
        except Exception as e:
            raise e
        return check_videos(url, fpath)


def convert_time(t):
    mins, secs = map(int, t.split(":"))
    millis = (secs + mins*60)*1000
    return millis
    
def split_video(fpath, start_time, end_time):
    start_time = convert_time(start_time)
    end_time = convert_time(end_time)
    sound = AudioSegment.from_mp3(fpath)
    sound = sound[start_time:end_time].fade_in(500).fade_out(500)
    split_path = os.path.split(fpath)
    fname, ext = split_path[-1].split(".")
    fpath = os.path.join(split_path[0], fname)
    new_path = fpath + "_{}_{}.{}".format(start_time, end_time, ext)
    sound.export(new_path, format=ext)
    return new_path

In [229]:
url = hulk_test
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(url, download=False)

In [231]:
info['duration']

188

In [225]:
hulk_path = download_video(hulk_test, ydl_opts, './')

In [226]:
hulk_path.split(".")

['', '/-HTsz4dIQwI_1561831506', 'mp3']

In [227]:
split_video(hulk_path, "1:18", "1:20")

'./-HTsz4dIQwI_1561831506_78000_80000.mp3'

In [208]:
convert_time("1:15")

75000

In [187]:
# test youtube-dl
test_embed = 'https://www.youtube.com/embed/P23PzmEpwk0?start=20&end=30'
test_full = 'https://www.youtube.com/watch?v=P23PzmEpwk0'
hulk_test = 'https://www.youtube.com/watch?v=-HTsz4dIQwI'


In [87]:
conn = sqlite3.connect('../data/discord.db')
c = conn.cursor()

In [106]:
def init_db(c):
    tables = c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [r[0] for r in tables]
    if 'voice_clips' not in tables:
        c.execute('''CREATE TABLE voice_clips
             (member_id integer PRIMARY KEY, voice_clips text)''')

In [118]:
def get_current_clips(member_id, c):
    row = c.execute('''select member_id, voice_clips from voice_clips where member_id = {}'''.format(member_id))
    row = c.fetchone()
    if row is None:
        return row
    return row[1]

In [133]:
def update_current_clips(member_id, voice_clips, c):
    c.execute('''replace into voice_clips (member_id, voice_clips) 
                 values ({}, "{}")'''.format(member_id, voice_clips))

In [ ]:
def download_video()

In [132]:
conn.commit()
init_db(c)
conn.commit()
update_current_clips(122, "zzz.mp3|test_clip_02.mp3", c)
conn.commit()
get_current_clips(122, c)

'zzz.mp3|test_clip_02.mp3'

In [122]:
c.execute('''replace into voice_clips (member_id, voice_clips) 
                values (10, "test")''')

In [72]:
# Create table
c.execute('''CREATE TABLE voice_clips
             (member_id integer PRIMARY KEY, voice_clips text)''')

In [57]:
conn.commit()

In [58]:
for row in c.execute('''select * from voice_clips'''):
    print(row)

(10, 'value2')


In [59]:
c.execute('''REPLACE ''')

OperationalError: incomplete input

In [62]:
conn.commit()

In [67]:
for row in c.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(row)

('voice_clips',)


In [232]:
"!clip https://www.youtube.com/watch?v=-HTsz4dIQwI 1:18 1:20".split()

['!clip', 'https://www.youtube.com/watch?v=-HTsz4dIQwI', '1:18', '1:20']

In [234]:
x, y, z = "!clip https://www.youtube.com/watch?v=-HTsz4dIQwI 1:18 1:20".split()

ValueError: too many values to unpack (expected 3)

In [235]:
l = []
l is None

False

In [236]:
def get_current_clip(member_id, c):
    row = c.execute('''select member_id, voice_clips from voice_clips where member_id = {}'''.format(member_id))
    row = c.fetchone()
    if row is None:
        return row
    return row[1]

In [241]:
print(get_current_clip(2, c))

None
